<table style="border: none" align="left">
   <tr style="border: none">
      <th style="border: none"><font face="verdana" size="5" color="black"><b>Predicting customer churn with IBM Watson Machine Learning</b></th>
      <th style="border: none"><img src="https://github.com/pmservice/customer-satisfaction-prediction/blob/master/app/static/images/ml_icon_gray.png?raw=true" alt="Watson Machine Learning icon" height="40" width="40"></th>
  <tr style="border: none">
       <th style="border: none"><img src="https://github.com/pmservice/wml-sample-models/blob/master/spark/customer-satisfaction-prediction/images/users_banner_2-03.png?raw=true" width="600" alt="Icon"> </th>
   </tr>
</table>

This notebook contains steps and code to develop a predictive model, and start scoring new data. This notebook introduces commands for getting data and for basic data cleaning and exploration, pipeline creation, model training, model persistance to Watson Machine Learning repository, model deployment, and scoring.

Some familiarity with Python is helpful. This notebook uses Python 2.7 and Apache® Spark 2.0.

You will use a data set, **Telco Customer Churn**, which details anonymous customers data of telecommunication company. Use the details of this data set to predict customer churn which is very critical to business as it's easier to retain existing customers rather than acquiring new ones.

## Learning goals

The learning goals of this notebook are:

-  Load a CSV file into an Apache® Spark DataFrame.
-  Explore data.
-  Prepare data for training and evaluation.
-  Create an Apache® Spark machine learning pipeline.
-  Train and evaluate a model.
-  Persist a pipeline and model in Watson Machine Learning repository.
-  Explore and visualize prediction result using the plotly package.
-  Deploy a model for batch scoring using Wastson Machine Learning API.


## Contents

This notebook contains the following parts:

1.	[Setup](#setup)
2.	[Load and explore data](#load)
3.	[Create spark ml model](#model)
4.	[Persist model](#persistence)
5.	[Predict locally and visualize](#visualization)
6.	[Deploy and score in a Cloud](#scoring)
7.	[Summary and next steps](#summary)

<a id="setup"></a>
## 1. Setup

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Create a [Watson Machine Learning Service](https://console.ng.bluemix.net/catalog/services/ibm-watson-machine-learning/) instance required for model storing and deployment (a free plan is offered).
- Create a [Cloud Object Storage Service](https://console.bluemix.net/catalog/infrastructure/cloud_object_storage/) instance required for storing scoring data (a free plan is offered).
- Create a [Apache Spark Service](https://console.bluemix.net/catalog/services/apache-spark/) instance required for batch scoring (a free plan is offered).
-  Make sure that you are using a Spark 2.0 kernel in notebook.

### Create the Telco Customer Churn Data Asset  

The Telco Customer Churn data is available on project's Github page.

1.  Download dataset from **Telco Customer Churn project** page [data folder](https://github.com/pmservice/wml-sample-models/tree/master/spark/customer-satisfaction-prediction/data) to local machine filesystem.
2.  After downloading the dataset to your local machine, read it into Spark DataFrame.

In [1]:
import wget

link_to_data = 'https://github.com/pmservice/wml-sample-models/raw/master/spark/customer-satisfaction-prediction/data/WA_Fn%20UseC_%20Telco%20Customer%20Churn.csv'
filename = wget.download(link_to_data)

print filename

trainingTweets.csv


<a id="load"></a>
## 2. Load and explore data

In this section you will load the data as an Apache® Spark DataFrame and perform a basic exploration. 

Once you see the file on **Files** section, you can insert it to your notebook by selecting **Insert SparkSession DataFrame**

In [2]:
spark = SparkSession.builder.getOrCreate()

df_data = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .option('inferSchema', 'true')\
  .option('nanValue', ' ')\
  .option('nullValue', ' ')\
  .load(filename)
df_data.take(5)

[Row(id=1, text=u'realdonaldtrump sse slashes offshore wind investmentwants british government to pay for its losses on these monstrosities   ', label=1),
 Row(id=2, text=u'cnnvideo hillaryclinton dammit why she missed', label=1),
 Row(id=3, text=u'realdonaldtrump thepodiumeffect without passion you dont have energy without energy you have nothing donald trump ', label=1),
 Row(id=4, text=u'sensanders the american higher education system is beginning to fall behind those in the rest of the world fairshot', label=1),
 Row(id=5, text=u'billburton realdonaldtrump as someone who demonstrated such bravery in combat i can see why youd have so many questions oh wait i mean stfu', label=0)]

Explore the loaded data by using the following Apache® Spark DataFrame methods:
-  print schema
-  count all records
-  show distribution of label classes

In [3]:
df_data.printSchema()

print "Number of fields: %3g" % len(df_data.schema)

root
 |-- id: integer (nullable = true)
 |-- text: string (nullable = true)
 |-- label: integer (nullable = true)

Number of fields:   3


As you can see, the data contains 21 fields. "Churn" field is the one we would like to predict (label).

In [4]:
print "Total number of records: " + str(df_data.count())

Total number of records: 5987


Data set contains 7043 records.

Now you will check if all records have complete data.

In [18]:
df_complete = df_data.dropna()

print "Number of records with complete data: %3g" % df_complete.count()

Number of records with complete data: 7032


You can see that there are some missing values you can investigate that all missing values are present in `TotalCharges` feature. We will use dataset with missing values removed for model training and evaluation.

Now you will inspect distribution of classes in label column.

In [19]:
df_complete.groupBy('Churn').count().show()

+-----+-----+
|Churn|count|
+-----+-----+
|   No| 5163|
|  Yes| 1869|
+-----+-----+



<a id="model"></a>
## 3. Create an Apache® Spark machine learning model

In this section you will learn how to prepare data, create an Apache® Spark machine learning pipeline, and train a model.

### 3.1: Prepare data

In this subsection you will split your data into: train, test and predict datasets.

In [20]:
(train_data, test_data, predict_data) = df_complete.randomSplit([0.8, 0.18, 0.02], 24)

print "Number of records for training: " + str(train_data.count())
print "Number of records for evaluation: " + str(test_data.count())
print "Number of records for prediction: " + str(predict_data.count())

Number of records for training: 5621
Number of records for evaluation: 1263
Number of records for prediction: 148


As you can see our data has been successfully split into three datasets: 

-  The train data set, which is the largest group, is used for training.
-  The test data set will be used for model evaluation and is used to test the assumptions of the model.
-  The predict data set will be used for prediction.

### 3.2: Create pipeline and train a model

In this section you will create an Apache® Spark machine learning pipeline and then train the model.

In the first step you need to import the Apache® Spark machine learning packages that will be needed in the subsequent steps.

In [21]:
from pyspark.ml.feature import StringIndexer, IndexToString, RFormula
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml import Pipeline, Model

In the following step, convert all the predictors to features vector and label feature convert to numeric.

In [22]:
lab = StringIndexer(inputCol = 'Churn', outputCol = 'label')
features = RFormula(formula = "~ gender + SeniorCitizen +  Partner + Dependents + tenure + PhoneService + MultipleLines + InternetService + OnlineSecurity + OnlineBackup + DeviceProtection + TechSupport + StreamingTV + StreamingMovies + Contract + PaperlessBilling + PaymentMethod + MonthlyCharges + TotalCharges - 1")

Next, define estimators you want to use for classification. Logistic Regression is used in the following example.

In [23]:
lr = LogisticRegression(maxIter = 10)

Let's build the pipeline now. A pipeline consists of transformers and an estimator.

In [24]:
pipeline_lr = Pipeline(stages = [features, lab , lr])

Now, you can train your Logistic Regression model by using the previously defined **pipeline** and **train data**.

In [25]:
model_lr = pipeline_lr.fit(train_data)

You can check your **model accuracy** now. To evaluate the model, use **test data**.

In [26]:
predictions = model_lr.transform(test_data)
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)

print("Test dataset:")
print("Accuracy = %3.2f" % accuracy)
print("Error = %3.2f" % (1.0 - accuracy))

Test dataset:
Accuracy = 0.81
Error = 0.19


You can tune your model now to achieve better accuracy. For simplicity of this example tuning section is omitted.

<a id="persistence"></a>
## 4. Persist model

In this section you will learn how to store your pipeline and model in Watson Machine Learning repository by using python client libraries.

First, you must import Watson Machine Learning repository client libraries.

**Note**: Apache® Spark 2.0 is required.

In [36]:
from repository.mlrepositoryclient import MLRepositoryClient
from repository.mlrepositoryartifact import MLRepositoryArtifact
from repository.mlrepository import MetaProps

Authenticate to Watson Machine Learning service on Bluemix.

**Action**: Put credentials from your instance of Watson Machine Learning service in Bluemix here.</div>

In [28]:
wml_credentials={
  "url": "https://ibm-watson-ml.mybluemix.net",
  "access_key": "***",
  "username": "***",
  "password": "***",
  "instance_id": "***"
}

In [29]:
ml_repository_client = MLRepositoryClient(wml_credentials['url'])
ml_repository_client.authorize(wml_credentials['username'], wml_credentials['password'])

Create model artifact (abstraction layer).

In [30]:
model_artifact = MLRepositoryArtifact(ml_artifact=model_lr, training_data=train_data, name="Customer Churn Model" )

**Tip**: The MLRepositoryArtifact method expects a trained model object, training data, and a model name. (It is this model name that is displayed by the Watson Machine Learning service).

### 4.1: Save pipeline and model

In this subsection you will learn how to save pipeline and model artifacts to your Watson Machine Learning instance.

In [31]:
saved_model = ml_repository_client.models.save(model_artifact)

Get saved model metadata from Watson Machine Learning.

**Tip**: Use *meta.available_props()* to get the list of available props.

In [32]:
print "\n".join(saved_model.meta.available_props())

inputDataSchema
evaluationMetrics
pipelineVersionHref
modelVersionHref
trainingDataRef
pipelineType
creationTime
lastUpdated
label
authorEmail
trainingDataSchema
authorName
version
modelType
runtime
evaluationMethod


In [33]:
print "creationTime: " + str(saved_model.meta.prop("creationTime"))
print "modelType: " + saved_model.meta.prop("modelType")
print "pipelineType: " + saved_model.meta.prop("pipelineType")
print "runtime: " + str(saved_model.meta.prop("runtime"))
print "label: " + saved_model.meta.prop("label")
print "modelVersionHref: " + saved_model.meta.prop("modelVersionHref")

creationTime: 2017-09-11 21:23:06.471000+00:00
modelType: sparkml-model-2.0
pipelineType: sparkml-pipeline-2.0
runtime: spark-2.0
label: Churn
modelVersionHref: https://ibm-watson-ml.mybluemix.net/v2/artifacts/models/6428ee3a-2fb5-4a62-b6ce-205284a28804/versions/f5a13ec2-65c7-4704-adc1-e74c82bbebf5


**Tip**: **modelVersionHref** is our model unique indentifier in the Watson Machine Learning repository.

In [34]:
published_model_ID = saved_model.uid

print "Model Id: " + str(published_model_ID)

Model Id: 6428ee3a-2fb5-4a62-b6ce-205284a28804


**Model Id** can be used to retrive latest model version from Watson Machine Learning instance.

### 4.2: Load model

In this subsection you will learn how to load back saved model from specified instance of Watson Machine Learning.

In [ ]:
loadedModelArtifact = ml_repository_client.models.get(saved_model.uid)

You can print for example model name to make sure that model has been loaded correctly.

In [ ]:
print str(loadedModelArtifact.name)

As you can see the name is correct. You have already learned how save and load the model from Watson Machine Learning repository.

<a id="visualization"></a>
## 5. Predict locally and visualize

In this section you will learn how to load data from batch scoring and visualize the prediction results with plotly package.

### 5.1: Make local prediction using previously loaded model and test data

In this subsection you will score *predict_data* data set.

In [40]:
predictions = loadedModelArtifact.model_instance().transform(predict_data)

Preview the results by calling the *show()* method on the predictions DataFrame.

In [41]:
predictions.show(5)

+----------+------+-------------+-------+----------+------+------------+-------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------------+----------------+--------------------+--------------+------------+-----+--------------------+-----+--------------------+--------------------+----------+
|customerID|gender|SeniorCitizen|Partner|Dependents|tenure|PhoneService|MultipleLines|InternetService|OnlineSecurity|OnlineBackup|DeviceProtection|TechSupport|StreamingTV|StreamingMovies|      Contract|PaperlessBilling|       PaymentMethod|MonthlyCharges|TotalCharges|Churn|            features|label|       rawPrediction|         probability|prediction|
+----------+------+-------------+-------+----------+------+------------+-------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------------+----------------+--------------------+--------------+------------+-----+----------------

By tabulating a count, you can see the split between labels.

In [43]:
predictions.select("prediction").groupBy("prediction").count().show(truncate=False)

+----------+-----+
|prediction|count|
+----------+-----+
|0.0       |114  |
|1.0       |34   |
+----------+-----+



### 5.2: Sample visualization of data with Plotly package

In this subsection you will explore prediction results with Plotly, which is an online analytics and data visualization tool.

**Example**: First, you need to install required packages. You can do it by running the following code. Run it only one time.

    !pip install plotly --user 
    !pip install cufflinks --user

In [47]:
import sys
import pandas
import plotly.plotly as py
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import cufflinks as cf
import plotly.graph_objs as go

init_notebook_mode(connected=True)
sys.path.append("".join([os.environ["HOME"]])) 

Plot a pie chart that shows the predicted churn split. !!! By most important features: Contract, MonthlyCharges !!! predicted vs observed churns.

In [67]:
cumulative_stats = predictions.groupby(['label']).count()

product_data = [go.Pie(
            labels=[cumulative_stats.select('label').collect()[x][0] for x in range(2)],
            values=[cumulative_stats.select('count').collect()[x][0] for x in range(2)],
    )]

product_layout = go.Layout(
    title='Label split',
)

fig = go.Figure(data=product_data, layout=product_layout)
iplot(fig)

With this data set, you might want to do some analysis of Mean Monthly Charges per churn class

In [84]:
y_data_plot = [predictions.groupby(['label']).mean().collect()[x][3] for x in range(2)]
x_data_plot = [cumulative_stats.select('label').collect()[x][0] for x in range(2)]

63.44144144144145

In [85]:
age_data = [go.Bar(
            y= x_data_plot,
            x= y_data_plot
            
    )]

age_layout = go.Layout(
    title='Mean Monthly Charges per churn class',
    xaxis=dict(
        title = "Churn",
        showline=False,),
    yaxis=dict(
        title = "Mean Monthly Charges",
        ),
)

fig = go.Figure(data=age_data, layout=age_layout)
iplot(fig)

Based on bar plot you created, you might make the following conclusion: The mean monthly charges for churn customers is higher than non-churn customers as you may expect.

<a id="scoring"></a>
## 6. Deploy model and score in a Cloud

In this section you will learn how to create batch deployment and to score a new data record by using the Watson Machine Learning REST API. 
For more information about REST APIs, see the [Swagger Documentation](http://watson-ml-api.mybluemix.net/).

#### 6.1 Save data for prediction to Object Storage

Bla bla bla ...

**Example**: First, you need to install required packages. You can do it by running the following code. Run it only one time.

    !pip install python-swiftclient --user

First, you must import Object Storage client libraries.

In [87]:
import swiftclient.client as swiftclient
import urllib

<div>**Action**: Put credentials from Object Storage Service in Bluemix here.</div>

In [ ]:
objectstorage_credentials = {
  "auth_url": "https://identity.open.softlayer.com",
  "project": "***",
  "projectId": "***",
  "region": "***",
  "userId": "***",
  "username": "***",
  "password": "***",
  "domainId": "***",
  "domainName": "***",
  "role": "***"
}

In [88]:
connection_userid = objectstorage_credentials.get("userId")
connection_projectid = objectstorage_credentials.get("projectId")
connection_password = objectstorage_credentials.get("password")
connection_region = objectstorage_credentials.get("region")
connection_authurl = objectstorage_credentials.get("auth_url")

Now you will create container in Object Storage and copy [dataset](https://raw.githubusercontent.com/pmservice/wml-sample-models/master/spark/customer-satisfaction-prediction/data/scoreInput.csv) for batch scoring from **Telco Customer Churn project**.

In [90]:
score_filename = "scoreInput.csv"
container_name = "batchjob"

In [91]:
conn = swiftclient.Connection(key = connection_password, authurl = connection_authurl + "/v3", auth_version = '3', os_options = {"project_id": connection_projectid, "user_id": connection_userid, "region": connection_region})
object_storage_info = conn.get_account()
if(container_name not in [x.get('name') for x in object_storage_info[1]]):
    conn.put_container(container = container_name)
container_info = conn.get_container(container_name)
if(score_filename not in [x.get('name') for x in container_info[1]]):
    scoring_data_connection = urllib.urlopen('https://raw.githubusercontent.com/pmservice/wml-sample-models/master/spark/customer-satisfaction-prediction/data/scoreInput.csv')
    conn.put_object(container = container_name, obj = score_filename, contents = scoring_data_connection, content_type = 'text/plain')
    scoring_data_connection.close()
    print "File '%s' successfully uploaded to container '%s'" % (score_filename, container_name)
else:
    print "File '%s' already exists in container '%s'" % (score_filename, container_name)
conn.close()

File 'scoreInput.csv' successfully uploaded to container 'batchjob'


### 6.2: Create model deployment

Now you can create a batch scoring endpoint. Execute the following sample code that uses the **published Model Id** value to create the scoring endpoint to the Bluemix repository.

#### Create access token to Watson Machine Learning Service

To work with the Watson Machine Leraning REST API you must generate an access token. To do that you can use the following sample code:

First, you must import standard python libraries.

In [92]:
import urllib3, requests, json

headers = urllib3.util.make_headers(basic_auth='{username}:{password}'.format(username=wml_credentials['username'], password=wml_credentials['password']))
url = '{}/v3/identity/token'.format(wml_credentials['url'])
response = requests.get(url, headers=headers)
mltoken = json.loads(response.text).get('token')

### 6.1: Create batch deployment

#### Get published_models url from instance details

In [93]:
endpoint_instance = wml_credentials['url'] + "/v3/wml_instances/" + wml_credentials['instance_id']
header = {'Content-Type': 'application/json', 'Authorization': 'Bearer ' + mltoken}

response_get_instance = requests.get(endpoint_instance, headers=header)

print response_get_instance
print response_get_instance.text

<Response [200]>
{"metadata":{"guid":"360c510b-012c-4793-ae3f-063410081c3e","url":"https://ibm-watson-ml.mybluemix.net/v3/wml_instances/360c510b-012c-4793-ae3f-063410081c3e","created_at":"2017-08-04T09:15:48.344Z","modified_at":"2017-09-11T21:23:06.628Z"},"entity":{"source":"Bluemix","published_models":{"url":"https://ibm-watson-ml.mybluemix.net/v3/wml_instances/360c510b-012c-4793-ae3f-063410081c3e/published_models"},"usage":{"expiration_date":"2017-10-01T00:00:00.000Z","computation_time":{"current":18},"model_count":{"limit":1000,"current":1},"prediction_count":{"current":29},"deployment_count":{"limit":1000,"current":7}},"plan_id":"0f2a3c2c-456b-40f3-9b19-726d2740b11c","status":"Active","organization_guid":"b0e61605-a82e-4f03-9e9f-2767973c084d","region":"us-south","account":{"id":"f52968f3dbbe7b0b53e15743d45e5e90","name":"Umit Cakmak's Account","type":"TRIAL"},"owner":{"ibm_id":"31000292EV","email":"umit.cakmak@pl.ibm.com","user_id":"43e0ee0e-6bfb-48fc-bcd8-d61e40d19253","country_cod

In [94]:
endpoint_published_models = json.loads(response_get_instance.text).get('entity').get('published_models').get('url')

print endpoint_published_models

https://ibm-watson-ml.mybluemix.net/v3/wml_instances/360c510b-012c-4793-ae3f-063410081c3e/published_models


Execute the following sample code that uses the published_models endpoint to get deployments url.

#### Get the list of published models

In [95]:
header = {'Content-Type': 'application/json', 'Authorization': 'Bearer ' + mltoken}
response_get = requests.get(endpoint_published_models, headers=header)

print response_get
print response_get.text

<Response [200]>
{"count":1,"resources":[{"metadata":{"guid":"6428ee3a-2fb5-4a62-b6ce-205284a28804","url":"https://ibm-watson-ml.mybluemix.net/v3/wml_instances/360c510b-012c-4793-ae3f-063410081c3e/published_models/6428ee3a-2fb5-4a62-b6ce-205284a28804","created_at":"2017-09-11T21:23:06.471Z","modified_at":"2017-09-11T21:23:06.988Z"},"entity":{"runtime_environment":"spark-2.0","author":{},"name":"Customer Churn Model","label_col":"Churn","training_data_schema":{"fields":[{"metadata":{},"type":"string","name":"customerID","nullable":true},{"metadata":{},"type":"string","name":"gender","nullable":true},{"metadata":{},"type":"integer","name":"SeniorCitizen","nullable":true},{"metadata":{},"type":"string","name":"Partner","nullable":true},{"metadata":{},"type":"string","name":"Dependents","nullable":true},{"metadata":{},"type":"integer","name":"tenure","nullable":true},{"metadata":{},"type":"string","name":"PhoneService","nullable":true},{"metadata":{},"type":"string","name":"MultipleLines",

#### Get published model deployment url

In [96]:
[endpoint_deployments] = [x.get('entity').get('deployments').get('url') for x in json.loads(response_get.text).get('resources') if x.get('metadata').get('guid') == saved_model.uid]

print endpoint_deployments

https://ibm-watson-ml.mybluemix.net/v3/wml_instances/360c510b-012c-4793-ae3f-063410081c3e/published_models/6428ee3a-2fb5-4a62-b6ce-205284a28804/deployments


#### Create batch deployment for published model

First import standard python library.

In [97]:
import base64

<div>**Action**:  Put your credentials from Apache Spark Service in Bluemix here</div>

In [98]:
spark_credentials = {
  "tenant_id": "s37a-8ea2eec65e9690-35c9a7ecf27a",
  "tenant_id_full": "9577bb8c-9986-42c1-b37a-8ea2eec65e96_4c55eb1c-d6fe-4f0a-9390-35c9a7ecf27a",
  "cluster_master_url": "https://spark.bluemix.net",
  "tenant_secret": "2942a6ca-ac77-4d63-805e-9f0e2aae716d",
  "instance_id": "9577bb8c-9986-42c1-b37a-8ea2eec65e96",
  "plan": "ibm.SparkService.PayGoPersonal"
}

In [99]:
encoded_spark_credentials = base64.b64encode(json.dumps({"credentials": spark_credentials, "version": "2.0"}))
result_filename = "result.csv"

In [101]:
header_batch = {'Content-Type': 'application/json', 'Authorization': "Bearer " + mltoken, 'X-Spark-Service-Instance': encoded_spark_credentials}
payload_batch = {"type":"batch", "name": "Customer Churn Prediction", "description": "Batch Deployment", "input": {"source": { "container": container_name, "filename": score_filename, "fileformat": "csv", "inferschema": 1, "firstlineheader": "true", "type": "bluemixobjectstorage"}, "connection": {"userid": connection_userid, "projectid": connection_projectid, "password": connection_password, "region": connection_region, "authurl": connection_authurl}}, "output": { "target": { "container": container_name, "filename": result_filename, "fileformat": "csv", "firstlineheader": "true", "type": "bluemixobjectstorage"}, "connection": {"userid": connection_userid, "projectid": connection_projectid, "password": connection_password, "region": connection_region, "authurl": connection_authurl}}}

response_batch = requests.post(endpoint_deployments, json=payload_batch, headers=header_batch)

print response_batch
print json.loads(response_batch.text).get("entity").get("status")

<Response [201]>
INITIALIZING


You can see that your job has been submited to Spark.

#### Monitor job execution

Here you can check status of your batch scoring. When status of Spark job is `FINNISHED` the results will be written to result.csv file in Object Storage.

In [104]:
header_monitor_job = {'Content-Type': 'application/json', 'Authorization': "Bearer " + mltoken, 'X-Spark-Service-Instance': encoded_spark_credentials}

response_monitor_job = requests.get(endpoint_deployments, headers=header_monitor_job)

print response_monitor_job
print response_monitor_job.text

<Response [200]>
{"count":1,"resources":[{"metadata":{"guid":"2756d942-75b2-4127-88a6-611cd211739e","url":"https://ibm-watson-ml.mybluemix.net/v3/wml_instances/360c510b-012c-4793-ae3f-063410081c3e/published_models/6428ee3a-2fb5-4a62-b6ce-205284a28804/deployments/2756d942-75b2-4127-88a6-611cd211739e","created_at":"2017-09-11T21:40:52.170Z","modified_at":"2017-09-11T21:41:12.399Z"},"entity":{"runtime_environment":"spark-2.0","name":"Customer Churn Prediction","description":"Batch Deployment","published_model":{"author":{},"name":"Customer Churn Model","url":"https://ibm-watson-ml.mybluemix.net/v3/wml_instances/360c510b-012c-4793-ae3f-063410081c3e/published_models/6428ee3a-2fb5-4a62-b6ce-205284a28804","guid":"6428ee3a-2fb5-4a62-b6ce-205284a28804","created_at":"2017-09-11T21:40:52.126Z"},"model_type":"sparkml-model-2.0","status":"COMPLETED","output":{"connection":{"projectid":"a574e627c6f34c67996c28f65aa675e0","userid":"2a92211c534a4c908dd0dd4e901036a2","region":"dallas","authurl":"https:/

**Preview scored data**

In this subsection you will load scored data.

Get data with scores from Object Storage Service in Bluemix.

In [105]:
conn = swiftclient.Connection(key = connection_password, authurl = connection_authurl + "/v3", auth_version = '3', os_options = {"project_id": connection_projectid, "user_id": connection_userid, "region": connection_region})

In [110]:
from StringIO import StringIO

In [111]:
results = conn.get_object(container_name, result_filename)
results_pdf = pandas.read_csv(StringIO(results[1]))

Preview the results by calling the *show()* method on the predictions in Pandas data frame.

In [113]:
results_pdf

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,Churn,features,label,rawPrediction,probability,prediction
0,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,Month-to-month,Yes,Electronic check,Yes,"(30,[2,3,4,5,6,8,10,12,14,16,18,20,22,24,25,28...",1,"[-0.6683985606091145,0.6683985606091145]","[0.3388555225599606,0.6611444774400395]",1
1,3638-WEABW,Female,0,Yes,No,58,Yes,Yes,DSL,No,...,No,Two year,Yes,Credit card (automatic),No,"(30,[3,4,5,7,9,10,13,14,17,18,20,23,24,28,29],...",0,"[3.94407920167498,-3.94407920167498]","[0.9809989882724504,0.019001011727549564]",0
2,8665-UTDHZ,Male,0,Yes,Yes,1,No,No phone service,DSL,No,...,No,Month-to-month,No,Electronic check,Yes,"(30,[0,4,9,10,13,14,16,18,20,22,25,28,29],[1.0...",1,"[-0.09214192316229042,0.09214192316229042]","[0.4769808032441,0.5230191967558999]",1
3,8773-HHUOZ,Female,0,No,Yes,17,Yes,No,DSL,No,...,Yes,Month-to-month,Yes,Mailed check,Yes,"(30,[2,4,5,6,9,10,12,14,16,19,21,22,24,26,28,2...",1,"[0.747504546477989,-0.747504546477989]","[0.6786347093927774,0.32136529060722263]",0
4,4080-IIARD,Female,0,Yes,No,13,Yes,Yes,DSL,Yes,...,No,Month-to-month,Yes,Electronic check,No,"(30,[3,4,5,7,9,11,13,14,17,19,20,22,24,25,28,2...",0,"[0.5917942689227103,-0.5917942689227103]","[0.6437767283180145,0.3562232716819854]",0
5,6575-SUVOI,Female,1,Yes,No,25,Yes,Yes,DSL,Yes,...,No,Month-to-month,Yes,Credit card (automatic),No,"(30,[1,3,4,5,7,9,11,12,14,17,19,20,22,24,28,29...",0,"[1.0950900449544492,-1.0950900449544492]","[0.749338997934473,0.25066100206552694]",0
6,7495-OOKFY,Female,1,Yes,No,8,Yes,Yes,Fiber optic,No,...,No,Month-to-month,Yes,Credit card (automatic),Yes,"(30,[1,3,4,5,7,8,10,13,14,16,18,20,22,24,28,29...",1,"[-0.7967750064589325,0.7967750064589325]","[0.31071579857499326,0.6892842014250067]",1
7,0731-EBJQB,Female,0,Yes,Yes,52,Yes,No,No,No internet service,...,No internet service,One year,Yes,Electronic check,No,"(30,[4,5,6,24,25,28,29],[52.0,1.0,1.0,1.0,1.0,...",0,"[3.4144782057040937,-3.4144782057040937]","[0.9681539637912697,0.031846036208730215]",0
8,1891-QRQSA,Male,1,Yes,Yes,64,Yes,Yes,Fiber optic,Yes,...,Yes,Two year,Yes,Bank transfer (automatic),No,"(30,[0,1,4,5,7,8,11,12,15,17,19,21,23,24,27,28...",0,"[2.837589689486215,-2.837589689486215]","[0.944673621382491,0.05532637861750898]",0
9,5919-TMRGD,Female,0,No,Yes,1,Yes,No,Fiber optic,No,...,No,Month-to-month,Yes,Electronic check,Yes,"(30,[2,4,5,6,8,10,12,14,16,19,20,22,24,25,28,2...",1,"[-0.7697688966511269,0.7697688966511269]","[0.3165291006791172,0.6834708993208828]",1


You can see the predictions for all the records in pandas data frame.

<a id="summary"></a>
## 7. Summary and next steps     

 You successfully completed this notebook! You learned how to use Apache Spark machine learning as well as Watson Machine Learning for model creation and deployment. Check out our _[Online Documentation](https://console.ng.bluemix.net/docs/services/PredictiveModeling/index.html?pos=2)_ for more samples, tutorials, documentation, how-tos, and blog posts. 

### Authors

**Lukasz Cmielowski**, PhD, is a Automation Architect and Data Scientist in IBM with a track record of developing enterprise-level applications that substantially increases clients' ability to turn data into actionable knowledge.

Copyright © 2017 IBM. This notebook and its source code are released under the terms of the MIT License.